Final workable with searchable download able tab work in google colab and also download in pc folder and its files view ,  tree view

In [15]:
# GOOGLE DRIVE – COLLAPSIBLE + LIVE SEARCH + DOWNLOAD (NO ERROR GUARANTEED)

from googleapiclient.discovery import build
from google.colab import auth, drive, files
from google.auth import default
from IPython.display import display, HTML
import time

drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

output_file = "/content/drive/MyDrive/Google_Drive_Tree_Final.html"
print("Scanning your Google Drive...")

# === Fetch all files ===
items = []
token = None
while True:
    res = service.files().list(q="trashed = false", spaces='drive',
                               fields='nextPageToken, files(id,name,mimeType,parents,size)',
                               pageSize=1000, pageToken=token).execute()
    items.extend(res.get('files', []))
    print(f"Fetched {len(items)} items...", end='\r')
    token = res.get('nextPageToken')
    if not token: break

print(f"\nTotal items: {len(items)}")
total_count = len(items)

# === Build hierarchy ===
by_id = {i['id']: i for i in items}
children = {}
for i in items:
    for p in i.get('parents', []):
        children.setdefault(p, []).append(i)

roots = [i for i in items if not i.get('parents') or i['parents'][0] not in by_id]

def sort_kids(kids):
    folders = sorted([x for x in kids if x['mimeType'] == 'application/vnd.google-apps.folder'],
                     key=lambda x: x['name'].lower())
    files   = sorted([x for x in kids if x['mimeType'] != 'application/vnd.google-apps.folder'],
                     key=lambda x: x['name'].lower())
    return folders + files

folder_id = 0
def next_id():
    global folder_id
    folder_id += 1
    return f"sub{folder_id}"

# === Recursive HTML builder ===
def build(item, prefix="", is_last=True):
    global folder_id
    name = item['name']
    fid = item['id']
    is_folder = item['mimeType'] == 'application/vnd.google-apps.folder'
    conn = "└── " if is_last else "├── "
    indent = prefix + ("    " if is_last else "│   ")

    size = ""
    if not is_folder and 'size' in item:
        s = int(item['size'])
        if s >= 1<<30: size = f" ({s/(1<<30):.2f} GB)"
        elif s >= 1<<20: size = f" ({s/(1<<20):.1f} MB)"
        else: size = f" ({s>>10} KB)"

    if is_folder:
        dom_id = next_id()
        line = f'{prefix}{conn}<span class="folder" onclick="toggle(\'{dom_id}\')">Folder <b>{name}</b></span><br>'
        line += f'<div id="{dom_id}" class="subtree">'
    else:
        dl = f"https://drive.google.com/uc?id={fid}&export=download"
        line = f'{prefix}{conn}File <a href="{dl}" target="_blank" class="file">{name}</a> '
        line += f'<span class="size">{size}</span> '
        line += f'<a href="{dl}" download class="dl">Download</a><br>'

    for i, kid in enumerate(sort_kids(children.get(fid, []))):
        line += build(kid, indent, i == len(children.get(fid, []))-1)
    if is_folder:
        line += '</div>'
    return line

# === Build full tree ===
tree_html = ""
for root in sort_kids(roots):
    if root['id'] == 'root' or root.get('name') == 'My Drive':
        tree_html += '<div class="root-title">My Drive</div>'
    else:
        tree_html += f'<div class="shared-title">Shared Drive: <b>{root["name"]}</b></div>'
    tree_html += build(root, "", True)

# === FINAL HTML (NO Python variables inside JS) ===
final_html = f"""<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Google Drive – Collapsible Tree</title>
<style>
  body {{font-family:Segoe UI,Arial,sans-serif;background:#f0f2f5;padding:20px;margin:0;}}
  h1 {{color:#1967d2;text-align:center;margin:15px;}}
  input {{width:100%;padding:16px;font-size:20px;border:3px solid #1967d2;border-radius:12px;box-sizing:border-box;}}
  .info {{text-align:center;margin:15px;font-size:18px;color:#555;}}
  .tree {{background:#fff;padding:30px;border-radius:16px;box-shadow:0 10px 30px rgba(0,0,0,0.15);
          font-family:Consolas,monospace;font-size:15px;line-height:1.8;}}
  .folder {{cursor:pointer;color:#1967d2;user-select:none;}}
  .folder:hover {{text-decoration:underline;}}
  .folder::before {{content:"Folder ";}}
  .file {{color:#1a0dab;text-decoration:none;}}
  .file:hover {{text-decoration:underline;}}
  .size {{color:#777;font-size:0.9em;}}
  .dl {{margin-left:12px;color:#d00;font-weight:bold;text-decoration:none;}}
  .dl:hover {{color:red;}}
  .subtree {{padding-left:22px;}}
  .root-title, .shared-title {{font-size:19px;font-weight:bold;color:#1967d2;margin:25px 0 10px;}}
  .hl {{background:#ffeb3b !important;padding:0 4px;border-radius:3px;}}
</style>
</head>
<body>
<h1>Google Drive – Full Collapsible Tree</h1>
<input type="text" id="search" placeholder="Live Search..." autofocus>
<div class="info">Total items: {total_count} • {time.strftime("%d %b %Y, %I:%M %p")}</div>
<div class="tree">{tree_html}</div>

<script>
function toggle(id) {{
  var el = document.getElementById(id);
  el.style.display = (el.style.display === 'none') ? 'block' : 'none';
}}

document.getElementById('search').oninput = function() {{
  var term = this.value.toLowerCase();
  var files = document.querySelectorAll('.file');
  var info = document.querySelector('.info');
  var found = 0;

  files.forEach(function(f) {{
    if (term && f.textContent.toLowerCase().includes(term)) {{
      f.classList.add('hl');
      found++;
    }} else {{
      f.classList.remove('hl');
    }}
  }});

  info.textContent = term
    ? "Found " + found + " result(s) for '" + term + "'"
    : "Total items: {total_count}";
}};
</script>
</body>
</html>"""

# === Save + Download + Preview ===
with open(output_file, "w", encoding="utf-8") as f:
    f.write(final_html)

print("\nHO GAYA BHAI – 100% ERROR-FREE!")
print("File saved →", output_file)
files.download(output_file)
display(HTML(final_html))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Scanning your Google Drive...
Fetched 100 items...
Total items: 100

HO GAYA BHAI – 100% ERROR-FREE!
File saved → /content/drive/MyDrive/Google_Drive_Tree_Final.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>